In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces practice

In [4]:
# https://codeforces.com/problemset/problem/1984/B
def large_addition(N: int) -> bool:
    if N < 10:
        return False
    else:
        N_str: str = str(N)
        length = len(N_str)
        for i in range(length):
            if i == 0 and N_str[i] != '1':
                return False
            elif i == length - 1 and N_str[i] == '9':
                return False
            elif i not in (0, length - 1) and N_str[i] == '0':
                return False

        return True

# Testing the solution
assert large_addition(1393938) == True
assert large_addition(200) == False
assert large_addition(10) == True

LC Virtual: 25th June 2024
https://leetcode.com/contest/smarking-algorithm-contest-3/

In [5]:
# Q1
# https://www.youtube.com/watch?v=x8T11yRLIyY
def minMoves(nums: list[int]) -> int:
    """
    Intution: Instead of increasing all n - 1 elements by 1 each time, try to decrement
    the max element to the smallest element. Relative distances between numbers are the same.
    """
    nums.sort()
    result = 0
    for i in range(len(nums) - 1, -1, -1):
        result += nums[i] - nums[0]
    return result

# Testing the solution
assert minMoves([1,2,3,4]) == 6
assert minMoves([1,2,3]) == 3

In [6]:
# Q2
def numberOfBoomerangs(points: list[list[int]]) -> int:
    def distance(pt1: list[int], pt2: list[int]) -> float:
        return math.sqrt(((pt1[0] - pt2[0]) ** 2) + ((pt1[1] - pt2[1]) ** 2))

    boommerangs: int = 0
    N = len(points)
    for i in range(N):
        curr = points[i]
        distances: dict[float, int] = dict()
        for j in range(N):
            if j != i:
                dist = distance(points[i], points[j])
                distances[dist] = distances.get(dist, 0) + 1

        for count in distances.values():
            boommerangs += math.perm(count, 2)

    return boommerangs

# Testing the solution
assert numberOfBoomerangs([[0,0],[1,0],[2,0]]) == 2
assert numberOfBoomerangs([[1,1],[2,2],[3,3]]) == 2
assert numberOfBoomerangs([[1,1]]) == 0

In [7]:
# Q3
def findMinArrowShots(points: list[list[int]]) -> int:
    # Sort by end and merge the start, end
    points.sort(key=lambda x: (x[1], x[0]))

    # Merge the intervals
    merged_intervals: list[tuple[int, int]] = []
    for start, end in points:
        if merged_intervals and merged_intervals[-1][1] >= start:
            merged_intervals[-1] = max(merged_intervals[-1][0], start), min(merged_intervals[-1][1], end)
        else:
            merged_intervals.append((start, end))

    return len(merged_intervals)

# Testing the solution
assert findMinArrowShots([[10,16],[2,8],[1,6],[7,12]]) == 2
assert findMinArrowShots([[1,2],[3,4],[5,6],[7,8]]) == 4
assert findMinArrowShots([[1,2],[2,3],[3,4],[4,5]]) == 2

In [8]:
# Q4
def numberOfArithmeticSlicesBrute(nums: list[int]) -> int:
    def is_valid(sub: list[int]) -> bool:
        if len(sub) < 3:
            return False
        else:
            diff = sub[1] - sub[0]
            for i in range(len(sub) - 1):
                if sub[i + 1] - sub[i] != diff:
                    return False
            return True

    count = 0
    def backtrack(i: int, subseq: list[int]):
        if i == N:
            count += is_valid(subseq)
        else:
            subseq.append(nums[i])
            backtrack(i + 1, subseq)
            subseq.pop()
            backtrack(i + 1, subseq)

    N = len(nums)
    backtrack(0, [])
    return count

In [9]:
# Upsolving
# https://www.youtube.com/watch?v=YIMwwT9JdIE
def numberOfArithmeticSlices(nums: list[int]) -> int:
    """
    Ending at index 'j' what is maximum number of arthimetic subsequences that I can form >= 2
    """
    N = len(nums)
    dp: list[collections.defaultdict[int, int]] = [collections.defaultdict(int) for _ in range(N)]
    sub_count = dp_sum = 0
    for j in range(N):
        for i in range(j):
            delta = nums[i] - nums[j]
            dp[j][delta] += dp[i][delta] + 1
            sub_count, dp_sum = sub_count + 1, dp_sum + dp[i][delta] + 1

    return dp_sum - sub_count

# Testing the solution
assert numberOfArithmeticSlices([2,4,6,8,10]) == 7
assert numberOfArithmeticSlices([7,7,7,7,7]) == 16

LC VC: June 26th 2024

In [10]:
# Q4: Pattern 132
# https://www.youtube.com/watch?v=q5ANAl8Z458
# https://leetcode.com/problems/132-pattern/submissions/1301126531/
def find132pattern(nums: list[int]) -> bool:
    N = len(nums)

    # Monotonic stack, storing min values till index - value, min pairs
    stack: list[tuple[int, int]] = []
    min_: int = nums[0]

    for n in nums:
        while stack and stack[-1][0] <= n:
            stack.pop()

        if stack and stack[-1][1] < n:
            return True

        stack.append((n, min_))
        min_ = min(min_, n)

    return False

# Testing the solution
assert find132pattern([1,2,3,4]) == False
assert find132pattern([3,1,4,2]) == True
assert find132pattern([-1,3,2,0]) == True
assert find132pattern([3,1,2]) == False

Balance a Binary Search tree

In [11]:
# https://leetcode.com/problems/balance-a-binary-search-tree/submissions/1301150197
def balanceBST(root: BinaryTreeNode) -> BinaryTreeNode|None:
    def createBalancedBST(nodes_: list[int]) -> BinaryTreeNode|None:
        N = len(nodes_)
        if N == 0:
            return None
        else:
            mid = N // 2
            root_: BinaryTreeNode = BinaryTreeNode(nodes_[mid])
            root_.left = createBalancedBST(nodes_[:mid])
            root_.right = createBalancedBST(nodes_[mid + 1:])
            return root_

    # 1. Retrieve all elements in-order traversal (left, root, right), this returns an array in sorted order
    curr: BinaryTreeNode = root
    nodes: list[int] = []
    stack: list[BinaryTreeNode] = []
    while True:
        while curr:
            stack.append(curr)
            curr = curr.left
        if stack:
            curr = stack.pop()
            nodes.append(curr.val)
            curr = curr.right
        else:
            break

    # 2. Create a BST from the nodes: Take the middle element as the root, create left and right subtree recursively
    return createBalancedBST(nodes)

# Testing the solution
print(balanceBST(BinaryTreeNode.from_array([1,None,2,None,3,None,4,None,None])).to_list())

[3, 2, 4, 1]


LC Practice

In [12]:
def subarraySum(nums: list[int], k: int) -> int:
    prefix: collections.defaultdict = collections.defaultdict(int)
    prefix[0], count, psum = 1, 0, 0
    for n in nums:
        psum += n
        count += prefix[psum - k]
        prefix[psum] += 1

    return count

# Testing the solution
assert subarraySum([1,2,3], 3) == 2
assert subarraySum([1,1,1], 2) == 2

In [13]:
# https://leetcode.com/problems/continuous-subarray-sum/submissions/1301461461/
def checkSubarraySum(nums: list[int], k: int) -> bool:
    N = len(nums)
    prefix: dict[int, int] = {0: -1}
    psum = 0
    for i in range(N):
        psum = (psum + nums[i]) % k
        if i - prefix.get(psum, i) >= 2:
            return True
        prefix[psum] = min(prefix.get(psum, i), i)

    return False

# Testing the solution
assert checkSubarraySum([23,2,4,6,7], 6) == True
assert checkSubarraySum([0], 1) == False
assert checkSubarraySum([23,2,6,4,7,13], 13) == False
assert checkSubarraySum([23,2,6,4,7,13,13], 13) == True
assert checkSubarraySum([1,0], 2) == False
assert checkSubarraySum([1,2,3,4], 6) == True

Check if array is sorted and rotated

In [14]:
def checkArraySortedAndRotated(nums: list[int]) -> bool:
    N = len(nums)
    count = 0
    for i in range(N - 1):
        count += nums[i + 1] < nums[i]
    return count == 0 or (count == 1 and nums[0] > nums[-1])

# Testing the solution
assert checkArraySortedAndRotated([3,4,5,1,2]) == True
assert checkArraySortedAndRotated([2,1,3,4]) == False
assert checkArraySortedAndRotated([3,2,1]) == False

All ancestors of a node in a DAG

In [15]:
# https://leetcode.com/problems/all-ancestors-of-a-node-in-a-directed-acyclic-graph/submissions/1304043339
def getAncestors(N: int, edges: list[list[int]]) -> list[list[int]]:
    @functools.cache
    def DFS(n: int) -> set[int]:
        ancestors: set[int] = set()
        for next_ in adjl[n]:
            ancestors.add(next_)
            for next_ancestor in DFS(next_):
                ancestors.add(next_ancestor)
        return ancestors

    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n2].append(n1)

    result: list[list[int]] = [[] for _ in range(N)]
    for i in range(N):
        result[i] = sorted(DFS(i))

    return result

# Testing the solution
assert getAncestors(8, [[0,3],[0,4],[1,3],[2,4],[2,7],[3,5],[3,6],[3,7],[4,6]]) == [[],[],[],[0,1],[0,2],[0,1,3],[0,1,2,3,4],[0,1,2,3]]
assert getAncestors(5, [[0,1],[0,2],[0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]) == [[],[0],[0,1],[0,1,2],[0,1,2,3]]

LC Contest: 30th June 2024

In [16]:
# Q1
def maxHeightOfTriangle(red: int, blue: int) -> int:
    @functools.cache
    def backtrack(i: int) -> tuple[int, int]:
        if i == 0:
            return 0, 0
        else:
            prev = backtrack(i - 1)
            if i % 2 == 0:
                return prev[0], prev[1] + i
            else:
                return prev[0] + i, prev[1]

    red, blue = sorted([red, blue])
    low, high = 1, 20
    while low <= high:
        mid = (low + high) // 2
        r, b = sorted(backtrack(mid))
        if r <= red and b <= blue:
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert maxHeightOfTriangle(10, 1) == 2
assert maxHeightOfTriangle(1, 1) == 1
assert maxHeightOfTriangle(2, 1) == 2
assert maxHeightOfTriangle(2, 4) == 3

In [17]:
# Q2
def maximumLength(nums: list[int]) -> int:
    """
    Either all consequetive element pairs should sum up to an even number or to an odd number

    If all elements are to sum to an even number: either all should be even or all numbers should be odd => max(odd_count, even_count)
    If all elements are to sum to an odd number: greedily check validity (consequetive elements should be odd-even or even-odd)
    """
    odd_sum_length = odd_count = even_count = 0
    prev = 0 if nums[0] % 2 else 1
    for i in range(len(nums)):
        if nums[i] % 2 == 0:
            even_count += 1
        else:
            odd_count += 1

        if nums[i] % 2 != prev:
            odd_sum_length, prev = odd_sum_length + 1, nums[i] % 2

    return max(odd_sum_length, odd_count, even_count)

# Testing the solution
assert maximumLength([1,2,3,4]) == 4
assert maximumLength([1,2,1,1,2,1,2]) == 6
assert maximumLength([1,3]) == 2

search-query-auto-complete: Striver A2Z: First question

In [18]:
class AutoCompleteSystem:
    def __init__(self, sentences: list[str], times: list[int]) -> None:
        # Store a dictionary of sentences along with their frequencies
        self.sentences: collections.defaultdict[str, int] = collections.defaultdict(lambda: 0)
        self.acc: str = ""
        for sentence, freq in zip(sentences, times):
            self.sentences[sentence] = freq

    def input(self, character: str) -> list[str]:
        if character != '#':
            self.acc += character
            result: list[tuple[int, str]] = []
            for sentence, freq in self.sentences.items():
                if sentence.startswith(self.acc):
                    result.append((-freq, sentence))
            result.sort()
            return list(map(lambda x: x[1], result[:3]))

        else:
            self.sentences[self.acc] += 1
            self.acc = ''
            return []

# Testing the solution - 1
temp = AutoCompleteSystem(["i love you", "island", "ironman", "i love geeksforgeeks"], [5,3,2,2])
assert temp.input("i") == ["i love you", "island","i love geeksforgeeks"]
assert temp.input(" ") == ["i love you", "i love geeksforgeeks"]
assert temp.input("a") == []
assert temp.input("#") == []
assert temp.input("i") == ['i love you', 'island', 'i love geeksforgeeks']
assert temp.input(" ") == ['i love you', 'i love geeksforgeeks', 'i a']
assert temp.input("a") == ['i a']
assert temp.input("m") == []
assert temp.input("#") == []
assert temp.input("i") == ['i love you', 'island', 'i love geeksforgeeks']
assert temp.input(" ") == ['i love you', 'i love geeksforgeeks', 'i a']
assert temp.input("a") == ['i a', 'i am']
assert temp.input("m") == ['i am']
assert temp.input(" ") == []

Patterns practice

In [19]:
def p1(N: int) -> None:
    for i in range(N):
        for j in range(N):
            print('*', end='')
        print()

# Testing the solution
p1(5)

*****
*****
*****
*****
*****


In [20]:
def p2(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print("*", end='')
        print()

# Testing the solution
p2(5)

*
**
***
****
*****


In [21]:
def p3(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(j + 1, end=' ')
        print()

# Testing the solution
p3(5)

1 
1 2 
1 2 3 
1 2 3 4 
1 2 3 4 5 


In [22]:
def p4(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(i + 1, end=' ')
        print()

# Testing the solution
p4(5)

1 
2 2 
3 3 3 
4 4 4 4 
5 5 5 5 5 


In [23]:
def p5(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p5(5)

*****
****
***
**
*


In [24]:
def p6(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print(j + 1, end='')
        print()

# Testing the solution
p6(5)

12345
1234
123
12
1


In [25]:
def p7(N: int) -> None:
    max_stars: int = (2 * N) - 1
    for i in range(1, N + 1):
        stars: int = (2 * i) - 1
        half = (max_stars - stars) // 2
        for j in range(half):
            print(" ", end='')
        for j in range(stars):
            print("*", end='')
        for j in range(half):
            print(" ", end='')
        print()

# Testing the solution
p7(5)

    *    
   ***   
  *****  
 ******* 
*********


In [26]:
def p8(N: int) -> None:
    max_stars: int = (2 * N) - 1
    for i in range(N, 0, -1):
        stars: int = (2 * i) - 1
        half = (max_stars - stars) // 2
        for j in range(half):
            print(" ", end='')
        for j in range(stars):
            print("*", end='')
        for j in range(half):
            print(" ", end='')
        print()

# Testing the solution
p8(5)

*********
 ******* 
  *****  
   ***   
    *    


In [27]:
def p9(N: int) -> None:
    p7(N)
    p8(N)

# Testing the solution
p9(5)

    *    
   ***   
  *****  
 ******* 
*********
*********
 ******* 
  *****  
   ***   
    *    


In [28]:
def p10(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print("*", end='')
        print()

    for i in range(N - 1, 0, -1):
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p10(5)

*
**
***
****
*****
****
***
**
*


In [29]:
def p11(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(1 if i % 2 == j % 2 else 0, end=" ")
        print()

# Testing the solution
p11(5)

1 
0 1 
1 0 1 
0 1 0 1 
1 0 1 0 1 


In [30]:
def p12(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            print(j if i >= j else " ", end="")
        for j in range(N, 0, -1):
            print(j if i >= j else " ", end="")
        print()

# Testing the solution
p12(5)

1        1
12      21
123    321
1234  4321
1234554321


In [31]:
def p13(N: int) -> None:
    counter = 1
    for i in range(1, N + 1):
        for j in range(i):
            print(counter, end=" ")
            counter += 1
        print()

# Testing the solution
p13(5)

1 
2 3 
4 5 6 
7 8 9 10 
11 12 13 14 15 


In [32]:
def p14(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(i):
            print(chr(ord('A') + j), end=" ")
        print()

# Testing the solution
p14(5)

A 
A B 
A B C 
A B C D 
A B C D E 


In [33]:
def p15(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print(chr(ord('A') + j), end=" ")
        print()

# Testing the solution
p15(5)

A B C D E 
A B C D 
A B C 
A B 
A 


In [34]:
def p16(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(i):
            print(chr(ord('A') + i - 1), end=" ")
        print()

# Testing the solution
p16(5)

A 
B B 
C C C 
D D D D 
E E E E E 


In [35]:
def p17(N: int) -> None:
    max_width = (2 * N) - 1
    for i in range(1, N + 1):
        width = (2 * i) - 1
        spaces = max_width - width
        for j in range(spaces // 2):
            print(" ", end="")
        for j in range(i):
            print(chr(ord('A') + j), end="")
        for j in range(i - 2, -1, -1):
            print(chr(ord('A') + j), end="")
        for j in range(spaces // 2):
            print(" ", end="")
        print()

# Testing the solution
p17(5)

    A    
   ABA   
  ABCBA  
 ABCDCBA 
ABCDEDCBA


In [36]:
def p18(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(N - i, N):
            print(chr(ord('A') + j), end=' ')
        print()

# Testing the solution
p18(5)

E 
D E 
C D E 
B C D E 
A B C D E 


In [37]:
def p19(N: int) -> None:
    max_width = 2 * N
    for i_ in range(2 * N):
        i = N - i_ if i_ < N else (i_ % N) + 1
        for j in range(i):
            print("*", end='')
        for j in range(max_width - (2 * i)):
            print(" ", end='')
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p19(5)

**********
****  ****
***    ***
**      **
*        *
*        *
**      **
***    ***
****  ****
**********


In [38]:
def p20(N: int) -> None:
    max_width = 2 * N
    for i_ in range(2 * N - 1):
        i = i_ + 1 if i_ < N else 2 * N - i_ - 1
        for j in range(i):
            print("*", end='')
        for j in range(max_width - 2 * i):
            print(" ", end='')
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p20(5)

*        *
**      **
***    ***
****  ****
**********
****  ****
***    ***
**      **
*        *


In [39]:
def p21(N: int) -> None:
    for i in range(N):
        for j in range(N):
            print("*" if i in (0, N - 1) or j in (0, N - 1) else " ", end='')
        print()

# Testing the solution
p21(5)

*****
*   *
*   *
*   *
*****


In [40]:
def p22(N: int) -> None:
    result: collections.deque[collections.deque[int]] = collections.deque([collections.deque([1])])
    for i in range(2, N + 1):
        for row in result:
            row.appendleft(i)
            row.append(i)
        result.appendleft(collections.deque([i for _ in range(2 * i - 1)]))
        result.append(collections.deque([i for _ in range(2 * i - 1)]))

    for row in result:
        for col in row:
            print(col, end=' ')
        print()

# Testing the solution
p22(3)

3 3 3 3 3 
3 2 2 2 3 
3 2 1 2 3 
3 2 2 2 3 
3 3 3 3 3 


Basic Math

In [41]:
def reverseDigits(N: int) -> int:
    NR, num = 0, abs(N)
    while num > 0:
        digit, num = num % 10, num // 10
        NR = (NR * 10) + digit
    return NR if N > 0 else -NR

# Testing the solution
assert reverseDigits(-123) == -321

In [42]:
def isPalindrome(N: int) -> bool:
    def reverse(num: int) -> int:
        numR = 0
        while num > 0:
            digit, num = num % 10, num // 10
            numR = (numR * 10) + digit
        return numR

    return N >= 0 and N == reverse(N)

# Testing the solution
assert isPalindrome(121) == True

In [43]:
def lcmAndGcd(n1: int , n2: int) -> tuple[int, int]:
    def primeFactors(n: int) -> dict[int, int]:
        factors: collections.defaultdict[int, int] = collections.defaultdict(int)
        curr = 2
        while curr <= n:
            if n % curr == 0:
                n //= curr
                factors[curr] += 1
            else:
                curr += 1
        return factors

    pf1, pf2 = primeFactors(n1), primeFactors(n2)
    factors: set[int] = {*pf1.keys(), *pf2.keys()}
    lcm = gcd = 1
    for factor in factors:
        min_, max_ = min(pf1[factor], pf2[factor]), max(pf1[factor], pf2[factor])
        if min_ != 0:
            gcd *= factor ** min_
        if max_ != 0:
            lcm *= factor ** max_

    return lcm, gcd
#
# Testing the solution
assert lcmAndGcd(5, 10) == (10, 5)
assert lcmAndGcd(14, 8) == (56, 2)
assert lcmAndGcd(8, 42) == (168, 2)

In [44]:
def lcmAndGcdOptimal(n1: int , n2: int) -> tuple[int, int]:
    """
    Euclidean algo: GCD remains same even if greater of two numbers is subtracted with the smaller number
    Continously subtract greater by smaller until we get a 0.
    """
    def gcd(n1: int, n2: int) -> int:
        while n1 != 0 and n2 != 0:
            n1, n2 = sorted([n1, n2])
            n1, n2 = n1, n2 - n1
        return max(n1, n2)

    gcd_ = gcd(n1, n2)
    lcm_ = (n1 * n2) // gcd_
    return lcm_, gcd_

# Testing the solution
assert lcmAndGcdOptimal(5, 10) == (10, 5)
assert lcmAndGcdOptimal(14, 8) == (56, 2)
assert lcmAndGcdOptimal(8, 42) == (168, 2)

In [45]:
def armstrong(N: int) -> bool:
    def countDigits(n: int) -> int:
        digits = 0
        while n > 0:
            digits, n = digits + 1, n // 10
        return digits

    total, n, K = 0, N, countDigits(N)
    while n > 0:
        digit, n = n % 10, n // 10
        total +=digit ** K

    return total == N

# Testing the solution
assert armstrong(371) == True
assert armstrong(123) == False

In [46]:
def printAllDivisors(N: int) -> list[int]:
    result: set[int] = set()
    i = 1
    while i * i <= N:
        if N % i == 0:
            result.add(i)
            result.add(N // i)
        i += 1

    return sorted(result)

# Testing the solution
assert printAllDivisors(12) == [1, 2, 3, 4, 6, 12]

In [47]:
def minJumps(arr: list[int], N: int) -> int:
    """
    Greedy Approach = Time: O(N), Space: O(1)

    Keep track of the range we can jump into with minimum number of steps
    For jump = 0, we can jump to index = 0 (left = right = 0)

    For each iteration through range, compute maximum range we can jump into.
    As we move with the next iteration, jumps = jumps + 1.
    Again recompute the next range
    """
    jumps = left = right = 0
    while right < N - 1:
        max_idx = 0
        for i in range(left, right + 1):
            max_idx = max(max_idx, i + arr[i])
        # For next iteration, we start with where we left off in the previous range (right + 1)
        jumps, left, right = jumps + 1, right + 1, max_idx

    return jumps

# Testing the solution
assert minJumps([1,3,5,8,9,2,6,7,6,8,9], 11) == 3

#### Basic Recursion

In [48]:
def sumOfSeries(n: int) -> int:
    """
    Figured this out by checking if there is a pattern by printing all results
    """
    sqrt = (n * (n + 1)) // 2
    return sqrt * sqrt

# Testing the solution
assert sumOfSeries(5) == 225
assert sumOfSeries(7) == 784

In [49]:
def factorialNumbers(N: int) -> list[int]:
    def factorial(i: int, curr: int) -> None:
        if curr > N:
            return
        else:
            result.append(curr)
            factorial(i + 1, curr * (i + 1))

    result: list[int] = []
    factorial(1, 1)
    return result

# Testing the solution
assert factorialNumbers(3) == [1, 2]
assert factorialNumbers(6) == [1, 2, 6]

In [50]:
def reverseArray(nums: list[int]):
    def reverse(i: int, j: int) -> None:
        if i >= j:
            return
        nums[i], nums[j] = nums[j], nums[i]
        reverse(i + 1, j - 1)

    N = len(nums)
    reverse(0, N - 1)
    return nums

# Testing the solution
assert reverseArray([1,2]) == [2,1]
assert reverseArray([1,2,3,4]) == [4,3,2,1]

In [51]:
def isPalindrome2(s: str) -> bool:
    def check(i: int, j: int) -> bool:
        if i >= j:
            return True
        elif not s[i].isalnum():
            return check(i + 1, j)
        elif not s[j].isalnum():
            return check(i, j - 1)
        else:
            return  s[i].lower() == s[j].lower() and check(i + 1, j - 1)

    N = len(s)
    return check(0, N - 1)

# Testing the solution
assert isPalindrome2("A man, a plan, a canal: Panama") == True
assert isPalindrome2("race a car") == False
assert isPalindrome2("0P") == False

In [52]:
def frequencyCount(arr: list[int], N: int, P: int) -> list[int]:
    """
    Similar to finding the missing duplicate number problem.
    """
    def backtrack(i: int) -> None:
        if i >= N:
            return
        elif arr[i] <= 0:
            arr[i] -= 1
        else:
            arr[i], next_ = -1, arr[i] - 1
            backtrack(next_)

    for i in range(N):
        if arr[i] > 0:
            arr[i], next_ = 0, arr[i] - 1
            backtrack(next_)

    for i in range(N):
        arr[i] = abs(arr[i])

    return arr

# Testing the solution
assert frequencyCount([2, 3, 2, 3, 5], 5, 5) == [0, 2, 2, 0, 1]
assert frequencyCount([3, 3, 3, 3], 4, 3) == [0, 0, 4, 0]
assert frequencyCount([8, 9], 2, 9)

In [53]:
def frequencyCountGFG(arr: list[int], N: int, P: int) -> list[int]:
    """
    Same idea but a diff approach: -ve values represent integers
    Instead of backtracking, we can swap with i until we hit a freq
    """
    for i in range(N):
        while arr[i] > 0:
            j = arr[i] - 1
            if j >= N:
                arr[i] = 0
            elif arr[j] <= 0: # j < N and arr[j] <= 0 (arr[j] is a freq)
                arr[j] -= 1
                arr[i] = 0
            else: # j < N and arr[j] > 0 (arr[j] is a number, we swap)
                arr[i], arr[j] = arr[j], arr[i]
                arr[j] = -1

    for i in range(N):
        arr[i] = abs(arr[i])

    return arr

assert frequencyCountGFG([2, 3, 2, 3, 5], 5, 5) == [0, 2, 2, 0, 1]
assert frequencyCountGFG([3, 3, 3, 3], 4, 3) == [0, 0, 4, 0]
assert frequencyCountGFG([8, 9], 2, 9)

LC Medium: https://leetcode.com/problems/frequency-of-the-most-frequent-element/description/

In [54]:
# Had to see the hint that is a sliding window problem
# https://leetcode.com/problems/frequency-of-the-most-frequent-element/submissions/1309518869
def maxFrequency(nums: list[int], k: int) -> int:
    nums.sort()
    N, max_freq, i, j, curr_max, cost = len(nums), 1, 0, 0, nums[0], 0
    while i < N and j < N:
        if cost <= k:
            max_freq = max(max_freq, j - i + 1)
            j += 1
            if j < N:
                cost += (nums[j] - curr_max) * (j - i)
                curr_max = max(curr_max, nums[j])
        else:
            cost -= curr_max - nums[i]
            i += 1

    return max_freq

# Testing the solution
assert maxFrequency([1,1,1,1,1,1,11,11,11,12,12,12,12,12,12,12,12], 10) == 11
assert maxFrequency([1], 10) == 1
assert maxFrequency([3,9,6], 2) == 1
assert maxFrequency([1,4,8,13], 5) == 2

In [55]:
# https://leetcode.com/problems/frequency-of-the-most-frequent-element/submissions/1309910554/
def maxFrequencyBetter(nums: list[int], k: int) -> int:
    """
    Continue expanding if cost <= k.
    If cost exceeds k, slide.
    """
    nums.sort()
    N, max_window, cost, curr_max = len(nums), 1, 0, nums[0]
    i, j = 0, 1
    while j < N:

        # Continue expanding always
        cost += (nums[j] - curr_max) * (j - i)
        curr_max = max(curr_max, nums[j])
        j += 1

        if cost <= k:
            max_window = max(max_window, j - i)
        else: # Shrink window
            cost -= curr_max - nums[i]
            i += 1

    return max_window

# Testing the solution
assert maxFrequencyBetter([1,1,1,1,1,1,11,11,11,12,12,12,12,12,12,12,12], 10) == 11
assert maxFrequencyBetter([1], 10) == 1
assert maxFrequencyBetter([3,9,6], 2) == 1
assert maxFrequencyBetter([1,4,8,13], 5) == 2

Sorting - 1

In [56]:
def ssort(nums: list[int]) -> list[int]:
    """
    Find the minimum and put it at the right place. Repeat for each index
    """
    N = len(nums)
    for i in range(N - 1):
        min_idx = i
        for j in range(i, N):
            if nums[j] < nums[min_idx]:
                min_idx = j
        nums[i], nums[min_idx] = nums[min_idx], nums[i]

    return nums

# Testing the solution
assert ssort([2,3,1,2,3,4]) == [1,2,2,3,3,4]

In [57]:
def bsort(nums: list[int]) -> list[int]:
    """
    Compares adjacent elements and places greatest element at the last of the window
    Best: O (N), Worst / Avg: O(N ^ 2)
    """
    N = len(nums)
    for i in range(N):
        swapped: bool = False
        for j in range(N - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j], nums[j + 1] = nums[j + 1], nums[j]
                swapped = True

        if not swapped:
            break

    return nums

# Testing the solution
assert bsort([2,3,1,2,3,4]) == [1,2,2,3,3,4]
assert bsort([1,2,3,4,5]) == [1,2,3,4,5]

In [58]:
def isort(nums: list[int]) -> list[int]:
    """
    Avg, Worst: O (N ^ 2), Best: O(N)
    For each element, make sure it is inserted correctly
    """
    N = len(nums)
    for i in range(N):
        j = i
        while j > 0 and nums[j] < nums[j - 1]:
            nums[j], nums[j - 1] = nums[j - 1], nums[j]
            j -= 1
    return nums

# Testing the solution
assert isort([2,3,1,2,3,4]) == [1,2,2,3,3,4]
assert isort([1,2,3,4,5]) == [1,2,3,4,5]

In [59]:
def isort2(nums: list[int]) -> list[int]:
    """
    Avg, Worst: O (N ^ 2), Best: O(N)
    For each element, make sure it is inserted correctly
    """
    N = len(nums)
    for i in range(1, N):
        curr, j = nums[i], i - 1
        while j >= 0 and nums[j] > curr:
            nums[j + 1] = nums[j]
            j -= 1
        nums[j + 1] = curr

    return nums

# Testing the solution
assert isort2([2,3,1,2,3,4]) == [1,2,2,3,3,4]
assert isort2([1,2,3,4,5]) == [1,2,3,4,5]

Code Gladiators Semi Final

In [60]:
# Q1
def max_rides_q1(arr1: list[int], N1: int, arr2: list[int], N2: int, money: int) -> int:
    i = j = rides = 0
    while i < N1 or j < N2:
        if i < N1 and arr1[i] <= money and (j >= N2 or arr1[i] <= arr2[j]):
            rides, i, money = rides + 1, i + 1, money - arr1[i]
        elif j < N2 and arr2[j] <= money and (i >= N1 or arr1[i] > arr2[j]):
            rides, j, money = rides + 1, j + 1, money - arr2[j]
        else:
            break

    return rides

# Testing the solution
assert max_rides_q1([5,3,6,8,10,20], 6, [4,4,4,6,7,10,4,8], 8, 20) == 5
assert max_rides_q1([10,20,30,40], 4, [5,3,9,10,2,3,6,9,15], 9, 60) == 9

Merge sort

In [61]:
def msort(arr: list[int]) -> list[int]:
    """
    Divide and merge
    Best, Avg, Worst: O(n log n)
    """
    def merge(arr1: list[int], arr2: list[int]) -> list[int]:
        N1, N2, i, j = len(arr1), len(arr2), 0, 0
        result: list[int] = []
        while i < N1 or j < N2:
            if j >= N2 or (i < N1 and arr1[i] <= arr2[j]):
                result.append(arr1[i])
                i += 1
            else:
                result.append(arr2[j])
                j += 1

        return result

    N = len(arr)
    if N == 1:
        return arr
    else:
        mid = N // 2
        left = msort(arr[:mid])
        right = msort(arr[mid:])
        return merge(left, right)

# Testing the solution
assert msort([5,4,3,2,1]) == [1,2,3,4,5]
assert msort([2,1]) == [1,2]
assert msort([1]) == [1]

In [62]:
def msortBetter(arr: list[int]) -> list[int]:
    def backtrack(left: int, right: int) -> None:
        if right - left <= 1:
            return
        else:
            mid = (left + right) // 2
            backtrack(left, mid)
            backtrack(mid, right)

            temp: list[int] = []
            i, j = left, mid
            while i < mid or j < right:
                if j >= right or (i < mid and arr[i] <= arr[j]):
                    temp.append(arr[i])
                    i += 1
                else:
                    temp.append(arr[j])
                    j += 1

            for idx in range(left, right):
                arr[idx] = temp[idx - left]

    N = len(arr)
    backtrack(0, N)
    return arr

# Testing the solution
for _ in range(50):
    arr = np.random.randint(0, 50, random.randint(1, 51)).tolist()
    msortBetter(arr)
    assert arr == sorted(arr)

Quick Sort

In [63]:
def qsort(arr: list[int]) -> list[int]:
    """
    Pick a pivot and place it at its correct position - smaller elements to the left, larger elements to the right of pivot
    Sort left & right halves recusively
    """
    N = len(arr)
    if N <= 1:
        return arr
    else:
        left, right, pivot = [], [], 0
        for i in range(N):
            if i != pivot:
                if arr[i] <= arr[pivot]:
                    left.append(arr[i])
                else:
                    right.append(arr[i])

        return qsort(left) + [arr[pivot]] + qsort(right)

# Testing the solution
for _ in range(50):
    arr = np.random.randint(0, 50, random.randint(1, 51)).tolist()
    assert qsort(arr) == sorted(arr)

In [64]:
def qsortBetter(arr: list[int]) -> list[int]:
    def partition(left: int, right: int, pivot: int) -> int:
        """
        Place two pointers, i = left, j = right
        Iterate i forward until we hit a number greater than arr[pivot]
        Iterate j backward until we hit a number lesser than or equal to arr[pivot]
        if i < j:
            Swap arr[i], arr[j]
        else:
            swap arr[pivot], arr[j]
            return j
        """
        i, j = left, right
        while i < j:
            while i < right and arr[i] <= arr[pivot]:
                i += 1
            while j > left and arr[j] > arr[pivot]:
                j -= 1
            if i < j:
                arr[i], arr[j] = arr[j], arr[i]

        arr[pivot], arr[j] = arr[j], arr[pivot]
        return j

    def backtrack(left: int, right: int) -> None:
        if left < right:
            pivot = partition(left, right, left)
            backtrack(left, pivot - 1)
            backtrack(pivot + 1, right)

    N = len(arr)
    backtrack(0, N - 1)
    return arr

# Testing the solution
for _ in range(50):
    arr = np.random.randint(0, 50, random.randint(1, 51)).tolist()
    qsortBetter(arr)
    assert arr == sorted(arr)

LC POTD

In [65]:
# https://leetcode.com/problems/pass-the-pillow/submissions/1311515636
def passThePillow(n: int, time: int) -> int:
    atFirst = (time // (n - 1)) % 2 == 0
    position = time % (n - 1)
    return position + 1 if atFirst else n - position

# Testing the solution
assert passThePillow(4, 5) == 2
assert passThePillow(3, 2) == 3
assert passThePillow(6, 5) == 6

In [66]:
def intersect(nums1: list[int], nums2: list[int]) -> list[int]:
    """
    Taking this approach since in one of the follow ups, its mentioned that the array is sorted
    """
    # Sort both the arrays
    nums1.sort()
    nums2.sort()

    N1, N2, i, j = len(nums1), len(nums2), 0, 0
    intersection: list[int] = []
    while i < N1 and j < N2:
        if nums1[i] < nums2[j]:
            i += 1
        elif nums1[i] > nums2[j]:
            j += 1
        else:
            intersection.append(nums1[i])
            i, j = i + 1, j + 1

    return intersection

# Testing the solution
assert intersect([1,2,2,1], [2,2]) == [2,2]
assert intersect([4,9,5], [9,4,9,8,4]) == [4,9]
assert intersect([4,9,5,4], [9,4,9,8,4]) == [4,4,9]

Codeforces Practice

In [67]:
# Drinks 800
def drinks(proportions: list[int], n_drinks: int) -> float:
    return (sum(proportions) * 100) / (100 * n_drinks)

# Testing the solution
assert round(drinks([50,50,100], 3), 2) == 66.67
assert round(drinks([0,25,50,75], 4), 2) == 37.5

In [68]:
# Substring and subsequence - 1200 
def substr_subsequence(text1: str, text2: str) -> int:
    N1, N2 = len(text1), len(text2)
    ans: int = N1 + N2
    for k in range(N2):
        i, j = 0, k
        while i < N1 and j < N2:
            if text1[i] == text2[j]:
                j += 1
            i += 1
        ans = min(ans, N1 + N2 - (j - k))

    return ans

# Testing the solution
assert substr_subsequence("aba", "cb") == 4
assert substr_subsequence("contest", "test") == 7
assert substr_subsequence("cde", "abcefg") == 7
assert substr_subsequence("cdaa", "cdca") == 6

LC Biweekly - 6th July, started at 9 PM IST

In [69]:
# Q1
def numberOfAlternatingGroups(colors: list[int]) -> int:
    N = len(colors)
    colors += colors[:2]
    groups = 0
    for i in range(N):
        if colors[i + 2] == colors[i] and colors[i] != colors[i + 1]:
            groups += 1
    return groups

# Testing the solution
assert numberOfAlternatingGroups([0,1,0,0,1]) == 3
assert numberOfAlternatingGroups([1,1,0]) == 1

In [70]:
# Q2
def maximumPoints(enemyEnergies: list[int], currentEnergy: int) -> int:
    """
    Sort by enemy energies.
    We can only consume energy once from an enemy - we do this from the most 'expensive' enemy
    We continue gaining points from the weaker guy
    """

    enemyEnergies.sort()
    N = len(enemyEnergies)
    points = 0
    while enemyEnergies:
        if enemyEnergies[0] <= currentEnergy:
            currentEnergy, points = currentEnergy % enemyEnergies[0], points + (currentEnergy // enemyEnergies[0])
        elif points > 0:
            currentEnergy += enemyEnergies.pop()
        else:
            break

    return points

# Testing the solution
assert maximumPoints([3,2,2], 2) == 3
assert maximumPoints([2], 10) == 5
assert maximumPoints([2], 1) == 0

Upsolving Q3

In [71]:
# Q3
# https://leetcode.com/problems/alternating-groups-ii/submissions/1311884953
def numberOfAlternatingGroupsII(colors: list[int], k: int) -> int:
    colors += colors[:k - 1]
    N = len(colors)
    i, j, prev, groups = 0, 0, int(not colors[0]), 0
    while j < N:
        if colors[j] != prev:
            prev = int(colors[j])
        else:
            i = j
        if j - i + 1 >= k:
            groups += 1

        j += 1

    return groups

# Testing the solution
assert numberOfAlternatingGroupsII([0,1,0,0,1,0,1], 6) == 2
assert numberOfAlternatingGroupsII([1,1,0,1], 4) == 0
assert numberOfAlternatingGroupsII([0,1,0,1,0], 3) == 3

Codeforces practice

In [72]:
def chewbacca(N: int) -> int:
    # Invert and add to stack
    stack: list[int] = []
    while N > 0:
        digit, N = N % 10, N // 10
        stack.append(digit if digit < 5 or (digit == 9 and N == 0) else 9 - digit)

    # Pop and accumulate to result
    result: int = 0
    while stack:
        result = (result * 10) + stack.pop()

    return result

# Testing the solution
assert chewbacca(4545) == 4444
assert chewbacca(27) == 22
assert chewbacca(9) == 9
assert chewbacca(919) == 910

LC Weekly - 7th July 2024

In [73]:
# Q1
def getEncryptedString(s: str, k: int) -> str:
    N = len(s)
    result: list[str] = []
    for i in range(N):
        result.append(s[(i + k) % N])
    return ''.join(result)

# Testing the solution
assert getEncryptedString("dart", 3) == "tdar"
assert getEncryptedString("aaa", 1) == "aaa"

In [74]:
# Q2
def validStrings(N: int) -> set[str]:
    def backtrack(acc: list[str]) -> None:
        if len(acc) == N:
            result.add(''.join(acc))
        else:
            if not acc or acc[-1] == '1':
                acc.append('0')
                backtrack(acc)
                acc.pop()
            acc.append('1')
            backtrack(acc)
            acc.pop()

    result: set[str] = set()
    backtrack([])
    return result

# Testing the solution
assert sorted(validStrings(3)) == ['010', '011', '101', '110', '111']

In [75]:
# Q4
def minimumCost(target: str, words: list[str], costs: list[int]) -> int:
    match_cache: dict[tuple[int, int], str] = dict()
    def match(i_: int, s_length: int, s: str) -> bool:
        if (i_, s_length) in match_cache:
            return s == match_cache[(i_, s_length)]
        elif i_ + s_length > target_length:
            return False
        else:
            i = i_
            for j in range(s_length):
                if i < target_length and s[j] == target[i]:
                    i += 1
                else:
                    return False

            match_cache[(i_, s_length)] = s
            return True

    @functools.cache
    def backtrack(i: int) -> float:
        if i == target_length:
            return 0
        else:
            min_cost: float = math.inf
            for cost, candidate in starts_with[target[i]]:
                candidate_length = len(candidate)
                if match(i, candidate_length, candidate):
                    min_cost = min(min_cost, cost + backtrack(i + candidate_length))
            return min_cost

    N_words = len(words)
    target_length = len(target)

    # Remove duplicates
    unique: collections.defaultdict[str, float] = collections.defaultdict(lambda: math.inf)
    for i in range(N_words):
        unique[words[i]] = min(unique[words[i]], costs[i])

    # Store the first character as a key
    starts_with: collections.defaultdict[str, list[tuple[float, str]]] = collections.defaultdict(list)
    for word, cost in unique.items():
        starts_with[word[0]].append((cost, word))

    # Sort by minimum cost
    for k in starts_with:
        starts_with[k].sort()

    # Backtrack and find the minimum cost
    min_cost = backtrack(0)
    return int(min_cost) if not math.isinf(min_cost) else -1

# Testing the solution
assert minimumCost("abcdef", ["abdef","abc","d","def","ef"], [100,1,1,10,5]) == 7
assert minimumCost("aaaa", ["z", "zz", "zzz"], [1, 10, 100]) == -1
assert minimumCost("aaaa", ["a", "z", "zz", "zzz"], [1, 1, 10, 100]) == 4
assert minimumCost("sgsipzma", ["s","s","g","ipzma"], [1,5,2,1]) == 5

Codeforces practice - Math

In [76]:
# Beautiful matrix
def beautiful_matrix(matrix: list[list[str]]) -> int:
    to_visit: collections.deque[tuple[int, int, int]] = collections.deque([(2, 2, 0)])
    visited: set[tuple[int, int]] = set()
    while to_visit:
        i, j, moves = to_visit.popleft()
        visited.add((i, j))
        if matrix[i][j] == '1':
            return moves
        else:
            for x, y in [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]:
                if 0 <= x < 5 and 0 <= y < 5 and (x, y) not in visited:
                    to_visit.append((x, y, moves + 1))
    return -1

# Testing the solution
assert beautiful_matrix([['0','0','0','0','0'], ['0','0','0','0','1'], ['0','0','0','0','0'], ['0','0','0','0','0'], ['0','0','0','0','0']]) == 3
assert beautiful_matrix([['0','0','0','0','0'], ['0','0','0','0','0'], ['0','1','0','0','0'], ['0','0','0','0','0'], ['0','0','0','0','0']]) == 1

In [77]:
def array_with_odd_sum(arr: list[int]) -> bool:
    N = len(arr)
    odd, even = functools.reduce(
            lambda initial, n: (initial[0] + int(n % 2 == 1), initial[1] + int(n % 2 == 0)),
            arr, (0, 0)
    )

    # If there are odd number of evens
    return (odd > 0 and even > 0) or (even == 0 and N % 2 == 1)

# Testing the solution
assert array_with_odd_sum([2,3]) == True
assert array_with_odd_sum([2,2,8,8]) == False
assert array_with_odd_sum([1,1,1,1]) == False
assert array_with_odd_sum([1,1,1]) == True

In [78]:
def floor_number(n: int, x: int) -> int:
    n = max(n - 2, 0)
    return math.ceil(n / x) + 1

# Testing the solution
assert floor_number(2, 100) == 1
assert floor_number(7, 3) == 3
assert floor_number(987, 13) == 77

Codeforces practice - Binary Search

In [79]:
def magic_powder(magic_powder: int, required: list[int], available: list[int]) -> int:
    def check(qty: int) -> bool:
        powder = magic_powder
        for req, avail in zip(required, available):
            powder -= max(0, (req * qty) - avail)
        return powder >= 0

    low = min(avail // req for req, avail in zip(required, available))
    high = low + magic_powder
    while low <= high:
        mid = (low + high) // 2
        if check(mid):
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert magic_powder(1, [2,1,4], [11,3,16]) == 4
assert magic_powder(3, [4,3,5,6], [11,12,14,20]) == 3

In [80]:
def pipeline(n: int, k: int) -> int:
    def check(i: int) -> bool:
        """
        Check if connecting the largest 'i' pipes would give flow >= n
        Sum of arithmetic progression: sn = n * ((a1 + an) / 2)
        """
        max_sum = (i * (k + k - i + 1) // 2) - i + 1
        return max_sum >= n

    low, high = 0, k
    while low <= high:
        mid = (low + high) // 2
        if check(mid):
            high = mid - 1
        else:
            low = mid + 1

    return low if low <= k else -1

# Testing the solution
assert pipeline(8, 4) == -1
assert pipeline(5, 5) == 1
assert pipeline(4, 3) == 2
assert pipeline(1, 4) == 0

In [81]:
def poisoned_dagger(N: int, hp: int, seconds: list[int]) -> int:
    def check(k: int) -> bool:
        damage = 0
        for i in range(N):
            damage += min(k, seconds[i + 1] - seconds[i])

        return damage >= hp

    # Update seconds for the sake of 'check'
    seconds.append(seconds[-1] + hp)

    # Binary search
    low, high = 1, hp
    while low <= high:
        mid = (low + high) // 2
        if check(mid):
            high = mid - 1
        else:
            low = mid + 1
    return low

# Testing the solution
assert poisoned_dagger(2, 5, [1, 5]) == 3
assert poisoned_dagger(3, 10, [2, 4, 10]) == 4
assert poisoned_dagger(5, 3, [1, 2, 4, 5, 7]) == 1
assert poisoned_dagger(4, 1000, [3, 25, 64, 1337]) == 470

Codeforces practice

In [82]:
def chocolate_theives(m: int) -> int:
    """
    Not clear on the upper bound aspect, led to TLE.
    Looks like we can fix UB to 8 (m + 1).
    """
    # Time: n ** 1/3
    def count_ways(n: int) -> int:
        count = 0
        k = 2
        while k ** 3 <= n:
            count += n // (k ** 3)
            k += 1
        return count

    # Binary search - log2(m)
    low, high = 8, 8 * (m + 1)
    while low <= high:
        mid = (low + high) // 2
        ways = count_ways(mid)
        if ways >= m:
            high = mid - 1
        else:
            low = mid + 1

    return low if count_ways(low) == m else -1

# Testing the solution
assert chocolate_theives(10) == -1
assert chocolate_theives(8) == 54
assert chocolate_theives(1) == 8

Striver CP - Prime, Seive, Prime Factorisation

In [83]:
def generate_primes(low: int, high: int) -> list[int]:
    """
    Return list of primes inclusive of range [low, high] using seive of Erastosthenes
    """
    soe: list[bool] = [True for i in range(high + 1)]
    soe[0], soe[1], i = False, False, 2
    while i * i <= high:
        if soe[i]:
            soe[i*i::i] = [False] * len(soe[i*i::i])
        i += 1

    return [i for i in range(low, high + 1) if soe[i]]

# Testing the solution
generate_primes(3, 11)

[3, 5, 7, 11]

In [84]:
def hulk(n: int) -> str:
    result: list[str] = []
    for i in range(n):
        result.append("I")
        result.append("hate" if i % 2 == 0 else "love")
        result.append("it" if i == n - 1 else "that")

    return " ".join(result)

# Testing the solution
assert hulk(1) == "I hate it"
assert hulk(2) == "I hate that I love it"
assert hulk(3) == "I hate that I love that I hate it"

Segmented Seive of Erastosthenes
Video Link: https://www.youtube.com/watch?v=MY0fXk-3BVQ

In [85]:
def segmented_seive_of_erastosthenes(low: int, high: int) -> list[int]:
    """
    1. Generate primes till sqrt(high)
    2. For each generated prime, repeat same logic - (i * i, end, i), instead of starting at i * i start at first position within range [low, high].
    """

    # Step 1
    sqrt = int(math.sqrt(high))
    seive: list[bool] = [True for i in range(sqrt + 1)]
    seive[0], seive[1], i = False, False, 2
    while i * i <= sqrt:
        if seive[i]:
            seive[i*i::i] = [False] * len(seive[i*i::i])
        i += 1

    primes_below: list[int] = [i for i in range(sqrt + 1) if seive[i]]

    # Step 2
    seive = [True for i in range(low, high + 1)]
    for prime in primes_below:
        # Always start at n * n and if that is smaller, we check for the 1st num in range
        start = max(prime * prime, math.ceil(low / prime) * prime)
        seive[start - low::prime] = [False] * len(seive[start - low::prime])

    primes: list[int] = [i for i in range(low, high + 1) if seive[i - low] and i != 1]

    return primes

# Testing the solution
assert segmented_seive_of_erastosthenes(100, 125) == [101, 103, 107, 109, 113]
assert segmented_seive_of_erastosthenes(1, 10) == [2, 3, 5, 7]

To count the number of primes available lesser than a number (approximately)

In [86]:
# Count of prime numbers <= n ~= ceil(n / ln(n))
def count_primes(n: int) -> int:
    return math.ceil(n / math.log(n))

# Testing the solution
assert count_primes(int(1e8)) >= 1e6
assert count_primes(int(1e9)) >= 1e7